<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# Naas - Send notification

**Tags:** #naas #emailbuilder #snippet #operations

**Author:** [Florent Ravenel](https://www.linkedin.com/in/ACoAABCNSioBW3YZHc2lBHVG0E_TXYWitQkmwog/)

**Description:** This notebook send a notification with your latest posts performance.

## Input

### Import libraries

In [ ]:
from naas_drivers import emailbuilder, gsheet
import naas
import os
import naas_data_product
from datetime import date, datetime, timedelta
from naas_drivers import naasauth

### Setup variables
**Inputs**
- `input_dir`: Input directory to retrieve file from.
- `file_content`: Name of the file to be retrieved.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `datalake_dir`: Datalake directory to retrieve plugin json file.

**Outputs**
- `email_to`: This variable is used for storing a list of email addresses that will receive the notification email.
- `subject`: This variable is used for storing the subject of the notification email.

In [ ]:
# Inputs
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "content-engine", date.today().isoformat())
file_content = "content"
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
sheet_name = "CONTENT"
datalake_dir = naas_data_product.OUTPUTS_PATH

# Outputs
email_to = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entity"), "emails") or []
email_subject = f"📲 Content Assistant - Email Update, {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

## Model

### Get assets

In [ ]:
image_url = get_image_asset(input_dir, "content_views_trend.png")
print("Image URL:", image_url)

plugin_url = naas.asset.find(os.path.join(datalake_dir, "plugins", "📲_content_assistant.json"))
print("Plugin URL:", plugin_url)

### Get data from Google Sheets spreadsheet

In [ ]:
df_input = pload(input_dir, file_content)    
if df_input is None:
    df_input = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)

# Filter on this week and last week
tw = date.today().strftime("W%W-%Y")
lw = (date.today() - timedelta(days=date.today().weekday() + 7)).strftime("W%W-%Y")
df_input = df_input[df_input["SCENARIO"].isin([tw, lw])]

# Display result
print("Rows:", len(df_input))
df_input.head(1)

### Prep values for email

In [ ]:
# Groupby weeks
df = df_input.groupby(["SCENARIO"], as_index=False).agg({"VIEWS": "sum", "LIKES": "sum", "COMMENTS": "sum", "SHARES": "sum"})
df = df.sort_values(by="SCENARIO").reset_index(drop=True)

def format_number(num):
    num = str("{:,.0f}".format(num)).replace(",", " ")
    return num

def create_var_text(df, column):
    value = df.loc[df.index[-1], column]
    value_n1 = df.loc[df.index[0], column]
    varv = value - value_n1
    return f"{format_number(value)} ({format_number(varv)})"

limit = (date.today() - timedelta(days=date.today().weekday())).isoformat()
views = create_var_text(df, "VIEWS")
likes = create_var_text(df, "LIKES")
comments = create_var_text(df, "COMMENTS")
shares = create_var_text(df, "SHARES")

# preview of what will be send by email:
print(f"{tw}:")
print(
    "\n\t👀 Impressions\t",
    views,
    "\n\t👍 Likes\t",
    likes,
    "\n\t💬 Comments\t",
    comments,
    "\n\t⏩ Shares\t",
    shares,
)

### Create Email brief

In [ ]:
def email_brief(
    views,
    likes,
    comments,
    shares,
    plugin_url,
    image_url,
):
    content = {
        'heading': "📲 Content Assistant",
        "txt_intro": (
            f"Hi there,<br><br>" f"Here is your content assistant email as of {date.today().isoformat()}."
        ),
        "title_1": emailbuilder.text(
            "Overview", font_size="27px", text_align="center", bold=True
        ),
        "image_1": emailbuilder.image(image_url),
        "text_1": emailbuilder.text(
            "Here are your content indicators stats vs last week:"
        ),
        "list_1": emailbuilder.list(
            [
                f"Views : <b>{views}</b>",
                f"Likes : <b>{likes}</b>",
                f"Comments : <b>{comments}</b>",
                f"Shares : <b>{shares}</b>",
            ]
        ),
        "button_1": emailbuilder.button(
            link=(f"https://naas.ai/chat/use?plugin_url={plugin_url}"),
            text="Start Chatting With Assistant",
            background_color="#181a1c",
        ),
        "footer_cs": emailbuilder.footer_company(naas=True),
    }

    email_content = emailbuilder.generate(display="iframe", **content)
    return email_content


email_content = email_brief(
    views,
    likes,
    comments,
    shares,
    plugin_url,
    image_url,
)

## Output

### Send the email

In [ ]:
if len(email_to) > 0:
    naas.notification.send(
        email_to=list(email_to.split(",")),
        subject=email_subject,
        html=email_content,
    )